# The JuMP ecosystem for mathematical optimization: Internals

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [23]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver
using Compat

## JuMP Containers

In [2]:
m = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(m, x[i=1:5, j=1:5])                        # Array      <=> @variable(m, x[i=1:5, j=1:5], container = Auto)
set_1 = Base.OneTo(5)
@variable(m, y[i=set_1, j=1:5])                      # Array
set_2 = 1:5
@variable(m, z1[i=1:5, j=set_2])                     # JuMPArray
@variable(m, z2[i=1:5, j=set_2], container = Array)  # Array
a = 1
@variable(m, zz1[i=1:5, j=a:5])                      # JuMPArray
@variable(m, zz2[i=1:5, j=a:5], container = Array)   # Array
set_3 = [:a, :b, :c]
@variable(m, w[i=set_2, j=set_3])                    # JuMPArray
@variable(m, t[i=set_2, j=1:i])                      # Dict
@variable(m, h[i = 1:5; j=isodd(i)])                 # Dict

Dict{Any,JuMP.VariableRef} with 3 entries:
  3 => h[3]
  5 => h[5]
  1 => h[1]

Auto chooses the tightest applicable container based on compile-time information only and is **type stable**

### `x[i=1:5, j=1:5]`

In [3]:
#@macroexpand @variable(m, x[i=1:5, j=1:5])  # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(x[i=1:5, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(1:5))...))

### `y[i=set_1, j=1:5], set_1 = Base.OneTo(5)`

In [4]:
#@macroexpand @variable(m, y[i=set_1, j=1:5]) # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(y[i=set_1, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_1 isa Main.Base.OneTo))) # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(set_1), (Main.length)(1:5))...)
    else  # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        Main.JuMP.JuMPArray((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(set_1), (Main.length)(1:5))...), set_1, 1:5)
    end)

### `z[i=1:5, j=set_2], set_2 = 1:5`

In [5]:
#@macroexpand @variable(m, z1[i=1:5, j=set_2])  # JuMPArray
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_2 isa Main.Base.OneTo))) # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
    else  # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        Main.JuMP.JuMPArray((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...), 1:5, set_2)
    end)

In [6]:
#@macroexpand @variable(m, z2[i=1:5, j=set_2], container = Array) # Array  
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

quote 
    $(Expr(:&&, :(Main.JuMP.validarrayindexset(set_2)))) || (Main.error)("Index set for array is not one-based interval.")
    (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
end

In [7]:
@code_typed JuMP.validarrayindexset(set_2)

CodeInfo(:(begin 
        return ((Core.getfield)(r, :start)::Int64 === 1)::Bool
    end))=>Bool

In [8]:
methods(JuMP.validarrayindexset)

# 3 methods for generic function "validarrayindexset":
validarrayindexset(::Base.OneTo) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:8
validarrayindexset(r::UnitRange) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:9
validarrayindexset(s) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:10

In [9]:
@code_typed JuMP.validarrayindexset(set_1)

CodeInfo(:(begin 
        return true
    end))=>Bool

### `zz[i=1:5, j=a:5], a = 1`

In [10]:
#@macroexpand @variable(m, zz1[i=1:5, j=a:5])  # JuMPArray
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(a:5 isa Main.Base.OneTo))) # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(a:5))...)
    else  # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        Main.JuMP.JuMPArray((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(a:5))...), 1:5, a:5)
    end)

In [11]:
#@macroexpand @variable(m, zz1[i=1:5, j=a:5], container = Array)   # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

quote 
    $(Expr(:&&, :(Main.JuMP.validarrayindexset(a:5)))) || (Main.error)("Index set for array is not one-based interval.")
    (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(a:5))...)
end

In [12]:
@code_typed JuMP.validarrayindexset(a:5)

CodeInfo(:(begin 
        return ((Core.getfield)(r, :start)::Int64 === 1)::Bool
    end))=>Bool

### Warning: Testing with JuMP.validarrayindexset may **NOT** be **type stable**

In [13]:
function getmyvariables(m,set)
    @variable(m,[set])
end

getmyvariables (generic function with 1 method)

In [14]:
function getmyvariableswithtesting(m,set)
    if JuMP.validarrayindexset(set)
        return @variable(m,[set],container=Array)
    else
        return @variable(m,[set])
    end
end

getmyvariableswithtesting (generic function with 1 method)

In [15]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariableswithtesting,Tuple{typeof(m),typeof(set_1)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::Array{JuMP.VariableRef,1}

In [16]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariableswithtesting,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::Union{Array{JuMP.VariableRef,1}, JuMP.JuMPArray{JuMP.VariableRef,1,Tuple{UnitRange{Int64}},Tuple{Dict{Int64,Int64}}}}

In [17]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariables,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::JuMP.JuMPArray{JuMP.VariableRef,1,Tuple{UnitRange{Int64}},Tuple{Dict{Int64,Int64}}}

`@variable` should always be type stable: 

In [18]:
function getmyvariablesarray(m,set)
    @variable(m,[set], container=Array)
end

getmyvariablesarray (generic function with 1 method)

In [19]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_1)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::Array{JuMP.VariableRef,1}

In [20]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::Array{JuMP.VariableRef,1}

In [21]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_3)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

  end::Union{}

In [22]:
@code_warntype getmyvariablesarray(m,set_3)

Variables:
  #self# <optimized out>
  m <optimized out>
  set <optimized out>
  #temp# <optimized out>
  685 <optimized out>
  ##684 <optimized out>

Body:
  begin 
      # meta: location /Users/jvielma/.julia/v0.6/JuMP/src/macros.jl # line 159:
      goto 4
      4: 
      (Base.throw)($(Expr(:new, :(Core.ErrorException), "Index set for array is not one-based interval.")))::Union{}
      # meta: pop location
  end::Union{}
